In [19]:
import numpy as np
import torch
import torch_geometric as pyg
import graphPINN
import math
import logging
from time import time
from tqdm.notebook import tqdm
from scipy.io import savemat, loadmat
import os

In [20]:
folder = "C:\\Users\\NASA\\Documents\\ML_checkpoints\\2023-07-07\\"
if not os.path.exists(f'{folder}extraps\\'):
    os.makedirs(f'{folder}extraps\\')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [21]:
k = 100

dataset = graphPINN.data.MHSDataset(f"E:\\scattered_data_v4_k={k}",k)


# trainset, validset, testset = torch.utils.data.random_split(dataset,[0.8, 0.1, 0.1],generator=torch.Generator().manual_seed(314))
trainset, validset, testset = torch.utils.data.random_split(dataset,[0.01, 0.005, 0.985],generator=torch.Generator().manual_seed(314))

In [22]:
# propdesign = [12,6,3]
propdesign = [12,3]
# convdesign = [18,9,6,3]
convdesign = [18,3]
params = 90

propkernel = graphPINN.KernelNN(propdesign, torch.nn.ReLU)
propgraph = graphPINN.BDPropGraph(propkernel)
convkernel = graphPINN.KernelNN(convdesign, torch.nn.ReLU)
convgraph = graphPINN.ConvGraph(convkernel)
model = pyg.nn.DataParallel(graphPINN.FullModel(propgraph, convgraph))
model.load_state_dict(torch.load(f'{folder}model_trainsize-32_k-50_params-90.pt'))
model = model.module.to(device)

TypeError: Expected state_dict to be dict-like, got <class 'graphPINN.core.FullModel'>.

In [23]:
params = 90
# params = 3132
model = torch.load(f'{folder}model_trainsize-32_k-100_params-90.pt')
model.to(device)
print(model)
# model = torch.load(f'{folder}_1680737885.339157model.pt').module

FullModel(
  (propgraph): BDPropGraph()
  (convgraph): ConvGraph()
)


In [24]:
set = 'train'

In [25]:
for j in tqdm(range(0,5)):
#     print(trainset[j].to_homogeneous().to(device))
    if set == 'train':
        data = trainset[j].to_homogeneous().to(device)
    elif set == 'validate':
        data = validset[j].to_homogeneous().to(device)
    elif set == 'test':
        data = testset[j].to_homogeneous().to(device)
    else:
        data = dataset.get(j).to_homogeneous().to(device)

    print(j,data.sharpnum.cpu().numpy()[0])
    pred = model(data)[0].detach()

    mat = {'Bml':pred.cpu().numpy(),
           'Btn':data.y.cpu().numpy(),
           'nodes':data.pos.cpu().numpy(),
           'pres':data.x[:,3:6].detach().cpu().numpy(),
           'sharp':data.sharpnum.cpu().numpy()[0]}
    savemat(f'{folder}\\extraps\\params-{params}_{set}{j}.mat',mat)

  0%|          | 0/5 [00:00<?, ?it/s]

0 7110
1 7581
2 7798
3 7081
4 7581


In [14]:
print(data.sharpnum.cpu().numpy()[0])

7085
